In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from matplotlib.image import imread
import numpy as np
%matplotlib inline


C:\Users\Wei Han Chen\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import os
import json
from skimage.color import rgb2gray
from skimage.transform import resize
import math
import random
import copy

# Get Image Data

In [5]:
image_data_directory = "Dataset/Color"
annotation_url = 'Dataset/annotation.json'
num_classes = 3

In [6]:
with open(annotation_url) as annotation:
    annotation_data = json.load(annotation)

In [7]:
colors = [
		[100.,  100.,  100.], 
		[100.,    0.,    0.],
		[150.,    0.,    0.],
		[200.,    0.,    0.],
		[255.,    0.,    0.],
		[100.,  100.,    0.],
		[150.,  150.,    0.],
		[200.,  200.,    0.],
		[255.,  255.,    0.],
		[  0.,  100.,   50.],
		[  0.,  150.,   75.],
		[  0.,  200.,  100.],
		[  0.,  255.,  125.],
		[  0.,   50.,  100.],
		[  0.,   75.,  150.],
		[  0.,  100.,  200.],
		[  0.,  125.,  255.],
		[100.,    0.,  100.],
		[150.,    0.,  150.],
        [200.,    0.,  200.],
        [255.,    0.,  255.]]

colors = np.divide(np.array(colors), 255)


In [8]:
def getImageData(image, pos, resize_ratio_x, resize_ratio_y, name):
    
    newpos_x = pos[:,0] / resize_ratio_x
    newpos_y = pos[:,1] / resize_ratio_y
    
    top_left_x = np.amin(newpos_x) - 2
    top_left_y = np.amin(newpos_y) - 2
    
    width = np.amax(newpos_x) - top_left_x + 2
    height = np.amax(newpos_y) - top_left_y + 2

    return top_left_x, top_left_y, width, height

In [9]:
def getCropImageData(image, pos, resize_ratio_x, resize_ratio_y,  name):
    
    newpos_x = pos[:,0] / resize_ratio_x
    newpos_y = pos[:,1] / resize_ratio_y
    
    top_left_x = np.amin(newpos_x) - 10
    top_left_y = np.amin(newpos_y) - 10
    
    width = np.amax(newpos_x) - top_left_x + 20
    height = np.amax(newpos_y) - top_left_y + 20


    return int(top_left_x), int(top_left_y), int(width), int(height)

In [10]:
def plotImageData(image, pos, resize_ratio_x, resize_ratio_y, name):
    plt.figure(figsize=(20,10))
#     image = rgb2gray(image)
    plt.imshow(image)
    
    newpos_x = pos[:,0] / resize_ratio_x
    newpos_y = pos[:,1] / resize_ratio_y
    
    top_left_x = np.amin(newpos_x) - 2
    top_left_y = np.amin(newpos_y) - 2
    
    width = np.amax(newpos_x) - top_left_x + 2
    height = np.amax(newpos_y) - top_left_y + 2

    
    rect = patches.Rectangle((top_left_x,top_left_y),width,height,linewidth=1,edgecolor='r',facecolor='none')
    
    plt.gca().add_patch(rect)
    
    plt.scatter(x=newpos_x, y=newpos_y, c=colors, s=10, alpha=0.7)
    plt.show()
    

In [11]:
def load_part_data(names, data_directory, annotation_data):

    old_x = 1080
    old_y = 1920
    reshaped_x = 126
    reshaped_y = 224
    cropped_resize_x = 32
    cropped_resize_y = 32
    
    allImages = np.empty((len(names), reshaped_x, reshaped_y, 3))
    allCroppedImages = np.empty((len(names), cropped_resize_x, cropped_resize_y, 3))
    allBoxs = np.empty((len(names), 4))
    allLabels = []
    
    for i in range(len(names)):
        name = names[i]
        label = 1 if name[-1] == 'L' else 2
        
        # get default but resized images
            
#         image = imread(data_directory + "/" + name[:-2] + ".jpg")
        path = data_directory + "/" + name[:-2] + ".jpg"
        
        image = tf.keras.preprocessing.image.load_img(path, target_size=(reshaped_x, reshaped_y))
        image = tf.keras.preprocessing.image.img_to_array(image)
        
        pos = np.array(annotation_data[name])

        resize_ratio_x = old_x / reshaped_x
        resize_ratio_y = old_y / reshaped_y
                                                      
#         resized_image = tf.image.resize_images(images=image,size=[reshaped_x, reshaped_y])
                                                      
#         resized_image = resize(image, (reshaped_x, reshaped_y, 3))
        
                                                      
                                                      
        top_x, top_y, width, height = getImageData(image, pos, 
                                        resize_ratio_x, resize_ratio_y, name)        
        boundingBox = np.array([top_x, top_y, width, height])
        
        newLabel = np.array([label])
        
        allImages[i] = image
        allBoxs[i] = boundingBox
        
        allLabels.append(newLabel) 
        

        # get cropped images
        
        top_x, top_y, width, height = getCropImageData(image, pos, resize_ratio_x, resize_ratio_y,  name)    
        
        cropped_image = image[top_y: top_y + height, top_x: top_x + width, :]
                
        
        
        cropped_image = resize(cropped_image / 255, (cropped_resize_x, cropped_resize_y, 3)) * 255
        
        allCroppedImages[i] = cropped_image
        
    
    print(allImages.shape)
    print(allBoxs.shape)
    
    return allImages, allCroppedImages, np.concatenate(allLabels), allBoxs
        


In [12]:
def load_data(data_directory, annotation_data):
    names = list(annotation_data.keys())

    print("Get train")
    train_data = np.array(names[:1000])
    X_train, X_crop_train, Y_train_label, Y_train_box = load_part_data(train_data, data_directory, annotation_data)
    
    
    print("Get valid")
    valid_data = np.array(names[2000:2500])
    X_val, X_crop_val, Y_val_label, Y_val_box = load_part_data(valid_data, data_directory, annotation_data)
    
    print("Get test")
    test_data = np.array(names[3000:3500])
    X_test, X_crop_test, Y_test_label, Y_test_box = load_part_data(test_data, data_directory, annotation_data)
    
    
    return (X_train, X_crop_train, Y_train_label, Y_train_box, 
            X_val, X_crop_val, Y_val_label, Y_val_box, 
            X_test, X_crop_test, Y_test_label, Y_test_box)

            

In [13]:
(X_train, X_crop_train, Y_train_label, Y_train_box, 
 X_val, X_crop_val, Y_val_label, Y_val_box, 
 X_test, X_crop_test, Y_test_label, Y_test_box) = load_data(image_data_directory, annotation_data)

Get train


C:\Users\Wei Han Chen\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


(1000, 126, 224, 3)
(1000, 4)
Get valid
(500, 126, 224, 3)
(500, 4)
Get test
(500, 126, 224, 3)
(500, 4)


In [14]:
print(X_train.shape)
print(X_crop_train.shape)
print(Y_train_label.shape)
print(Y_train_box.shape)

print(X_val.shape)
print(X_crop_val.shape)
print(Y_val_label.shape)
print(Y_val_box.shape)

print(X_test.shape)
print(X_crop_test.shape)
print(Y_test_label.shape)
print(Y_test_box.shape)

(1000, 126, 224, 3)
(1000, 32, 32, 3)
(1000,)
(1000, 4)
(500, 126, 224, 3)
(500, 32, 32, 3)
(500,)
(500, 4)
(500, 126, 224, 3)
(500, 32, 32, 3)
(500,)
(500, 4)


# Image Model

image data will have size N x W x H x C

N = number of images

W = width

H = height

C = RGB



In [13]:
def run_model(session, predict, loss_val, Xd, yd,
              epochs=1, batch_size=64, print_every=100,
              training=None, plot_losses=False):
    # have tensorflow compute accuracy
    correct_prediction = tf.equal(tf.argmax(predict,1, output_type=tf.int32), Y_label)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [mean_loss,correct_prediction,accuracy]
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%Xd.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         Y_label: yd[idx],
                         is_training: training_now }
            # get batch size
            actual_batch_size = yd[idx].shape[0]
            
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            loss, corr, _ = session.run(variables,feed_dict=feed_dict)
            
            
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                      .format(iter_cnt,loss,np.sum(corr)/actual_batch_size))
            iter_cnt += 1
        total_correct = correct/Xd.shape[0]
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss,total_correct

In [14]:
def my_model(X, y, is_training):
    convW1 = tf.get_variable('convW1', [5,5,3,30])
    convB1 = tf.get_variable('convB1', [30])
    convW2 = tf.get_variable('convW2', [3,3,30,60])
    convB2 = tf.get_variable('convB2', [60])
    convW3 = tf.get_variable('convW3', [5,5,60, 120])
    convB3 = tf.get_variable('convB3', [120]) 
    convW4 = tf.get_variable('convW4', [5,5,120, 100])
    convB4 = tf.get_variable('convB4', [100]) 
    convW5 = tf.get_variable('convW5', [8,8, 100, 3])
    convB5 = tf.get_variable('convB5', [3]) 
    
    print(X.shape)
    
    a1 = tf.nn.conv2d(X, convW1, strides=[1,1,1,1], padding="SAME") + convB1
    
    
    h1_batched = tf.layers.batch_normalization(a1, center=True, scale=True, training=is_training)
    
    
    h1 = tf.nn.elu(h1_batched)
    h1_pooled = tf.nn.max_pool(h1, [1,2,2,1], [1,2,2,1], padding="VALID")
    
    
    
    a2 = tf.nn.conv2d(h1_pooled, convW2, strides=[1,1,1,1], padding="SAME") + convB2
    h2_batched = tf.layers.batch_normalization(a2, center=True, scale=True, training=is_training)
    h2 = tf.nn.elu(h2_batched)
    h2_pooled = tf.nn.max_pool(h2, [1,2,2,1], [1,2,2,1], padding="VALID")
    
    
    a3 = tf.nn.conv2d(h2_pooled, convW3, strides=[1,1,1,1], padding="SAME") + convB3
    h3_batched = tf.layers.batch_normalization(a3, center=True, scale=True, training=is_training)
    h3 = tf.nn.elu(h3_batched)
    h3_pooled = tf.nn.max_pool(h3, [1,2,2,1], [1,2,2,1], padding="VALID")
    
    a4 = tf.nn.conv2d(h3_pooled, convW4, strides=[1,1,1,1], padding="SAME") + convB4
    h4_batched = tf.layers.batch_normalization(a4, center=True, scale=True, training=is_training)
    h4 = tf.nn.elu(h4_batched)
    h4_pooled = tf.nn.max_pool(h4, [1,2,2,1], [1,2,2,1], padding="VALID")
            
    a5 = tf.nn.conv2d(h4_pooled, convW5, strides=[1,1,1,1], padding="VALID") + convB5
    
    h5_batched = tf.layers.batch_normalization(a5, center=True, scale=True, training=is_training)
    h5 = tf.nn.elu(h5_batched)
    
    output = tf.reshape(h5, (-1,3))
        
    return output

In [15]:
# remove old variables

tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, 130, 130, 3])
Y_label = tf.placeholder(tf.int32, [None])
is_training = tf.placeholder(tf.bool)

print(X.shape)

(?, 130, 130, 3)


In [16]:
# y_out = my_model(X, Y_label, is_training)
# mean_loss = tf.reduce_mean(tf.losses.hinge_loss(tf.one_hot(Y_label, 3), logits=y_out))
# optimizer = tf.train.RMSPropOptimizer(1e-2)
# optimizer.minimize(mean_loss)


# # batch normalization in tensorflow requires this extra dependency
# extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
# with tf.control_dependencies(extra_update_ops):
#     train_step = optimizer.minimize(mean_loss)

In [17]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     print('Training')
#     run_model(sess, y_out, mean_loss, X_crop_train, Y_train_label ,10, 64, 100,train_step, True)
#     print('Validation')
#     run_model(sess, y_out, mean_loss, X_crop_val, Y_val_label , 1 , 64)

# Testing Tensorflow / Keras, Pretrained model

array([[[-0.99789544, -0.99786669, -0.99809666],
        [-0.99789512, -0.99786436, -0.99811042],
        [-0.99817462, -0.99814386, -0.99838992],
        ...,
        [-0.99569396, -0.99554018, -0.99572472],
        [-0.99568793, -0.99553414, -0.99571869],
        [-0.99594677, -0.99580303, -0.99597551]],

       [[-0.99886452, -0.99883577, -0.99906574],
        [-0.99888731, -0.99885655, -0.99910261],
        [-0.99904204, -0.99901128, -0.99925734],
        ...,
        [-0.99569222, -0.99553844, -0.99572298],
        [-0.99568261, -0.99552882, -0.99571336],
        [-0.99594179, -0.99579806, -0.99597054]],

       [[-0.99946598, -0.99943724, -0.99966721],
        [-0.99942862, -0.99939786, -0.99964392],
        [-0.99939916, -0.9993684 , -0.99961446],
        ...,
        [-0.99568391, -0.99553012, -0.99571467],
        [-0.99565891, -0.99550513, -0.99568967],
        [-0.99592631, -0.99578258, -0.99595506]],

       ...,

       [[-0.99638625, -0.99618503, -0.99664497],
        [-0

In [35]:
X_crop_train.shape

(1000, 32, 32, 3)

In [27]:
imageInput = tf.keras.Input(shape=(32, 32, 3))

In [28]:
model = tf.keras.applications.VGG16(input_tensor=imageInput, include_top=False, weights="imagenet")


In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [30]:
last_layer = model.get_layer('block5_pool').output

In [31]:
flatten = tf.keras.layers.Flatten()(last_layer)
out = tf.keras.layers.Dense(num_classes, activation="softmax", name="output")(flatten)

In [32]:
custom_vgg_model = tf.keras.Model(imageInput, out)

In [33]:
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [34]:
for layer in custom_vgg_model.layers[:-1]:
    layer.trainable = False
custom_vgg_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [36]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    train_y = np.zeros((len(Y_train_label), num_classes))
    val_y = np.zeros((len(Y_val_label), num_classes))
    test_y = np.zeros((len(Y_test_label), num_classes))
    
    train_y[np.arange(len(Y_train_label)), Y_train_label] = 1
    val_y[np.arange(len(Y_val_label)), Y_val_label] = 1
    test_y[np.arange(len(Y_test_label)), Y_test_label] = 1
    
    
    test1 = tf.keras.applications.vgg16.preprocess_input(X_crop_train)
    test2 = tf.keras.applications.vgg16.preprocess_input(X_crop_val)
    test3 = tf.keras.applications.vgg16.preprocess_input(X_crop_test)
    
    custom_vgg_model.fit(test1, train_y, batch_size=32,epochs=40, 
                         verbose=1, validation_data=(test2, val_y ))
    
    score = custom_vgg_model.evaluate(test3, test_y)
    print("Test Loss: ", score[0])
    print("Test Accu:", score[1])

Train on 1000 samples, validate on 500 samples
Epoch 1/40
1000/1000 [==============================] - 3s 3ms/step - loss: 0.7141 - acc: 0.5740 - val_loss: 0.6676 - val_acc: 0.5000
Epoch 2/40
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6371 - acc: 0.6640 - val_loss: 0.6158 - val_acc: 0.9000
Epoch 3/40
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6042 - acc: 0.8000 - val_loss: 0.5785 - val_acc: 0.9160
Epoch 4/40
1000/1000 [==============================] - 3s 3ms/step - loss: 0.5710 - acc: 0.9010 - val_loss: 0.5526 - val_acc: 0.9340
Epoch 5/40
1000/1000 [==============================] - 3s 3ms/step - loss: 0.5490 - acc: 0.8860 - val_loss: 0.5229 - val_acc: 0.9480
Epoch 6/40
1000/1000 [==============================] - 3s 3ms/step - loss: 0.5146 - acc: 0.9230 - val_loss: 0.5135 - val_acc: 0.8540
Epoch 7/40
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4946 - acc: 0.9330 - val_loss: 0.4738 - val_acc: 0.9580
Epoch 8/40
1000

# Keras Inception model

In [15]:
vgg19InputImage = tf.keras.Input(shape=(32,32,3))

In [16]:
vgg19_model = tf.keras.applications.VGG19(input_tensor=vgg19InputImage,
                                                include_top=False, weights="imagenet")

In [17]:
vgg19_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [18]:
vgg19_last = vgg19_model.get_layer("block5_pool").output

In [19]:
vgg19_flatten = tf.keras.layers.Flatten(name="vgg19_flatten")(vgg19_last)
vgg19_output = tf.keras.layers.Dense(num_classes, activation="softmax", name="output")(vgg19_flatten)

In [20]:
custom_vgg19_model = tf.keras.Model(vgg19InputImage, vgg19_output)

In [21]:
custom_vgg19_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [22]:
for layer in custom_vgg19_model.layers[:-1]:
    layer.trainable = False
custom_vgg19_model.compile(loss="categorical_hinge", optimizer="adam", metrics=["accuracy"])


In [23]:
custom_vgg19_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [26]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    train_y = np.zeros((len(Y_train_label), num_classes))
    val_y = np.zeros((len(Y_val_label), num_classes))
    test_y = np.zeros((len(Y_test_label), num_classes))
    
    train_y[np.arange(len(Y_train_label)), Y_train_label] = 1
    val_y[np.arange(len(Y_val_label)), Y_val_label] = 1
    test_y[np.arange(len(Y_test_label)), Y_test_label] = 1
    
    test1 = tf.keras.applications.vgg19.preprocess_input(X_crop_train)
    test2 = tf.keras.applications.vgg19.preprocess_input(X_crop_val)
    test3 = tf.keras.applications.vgg19.preprocess_input(X_crop_test)

    custom_vgg19_model.fit(test1, train_y, batch_size=20,epochs=50, 
                         verbose=1, validation_data=(test2, val_y ))
    
    score = custom_vgg19_model.evaluate(test3, test_y)
    print("Test Loss: ", score[0])
    print("Test Accu:", score[1])

Train on 1000 samples, validate on 500 samples
Epoch 1/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.9986 - acc: 0.5510 - val_loss: 0.9961 - val_acc: 0.5000
Epoch 2/50
1000/1000 [==============================] - 5s 5ms/step - loss: 0.9953 - acc: 0.4970 - val_loss: 0.9921 - val_acc: 0.5000
Epoch 3/50
1000/1000 [==============================] - 5s 5ms/step - loss: 0.9922 - acc: 0.4900 - val_loss: 0.9878 - val_acc: 0.5000
Epoch 4/50
1000/1000 [==============================] - 5s 5ms/step - loss: 0.9885 - acc: 0.4900 - val_loss: 0.9835 - val_acc: 0.5000
Epoch 5/50
1000/1000 [==============================] - 5s 5ms/step - loss: 0.9845 - acc: 0.4900 - val_loss: 0.9784 - val_acc: 0.5000
Epoch 6/50
1000/1000 [==============================] - 5s 5ms/step - loss: 0.9801 - acc: 0.4900 - val_loss: 0.9737 - val_acc: 0.5000
Epoch 7/50
1000/1000 [==============================] - 5s 5ms/step - loss: 0.9758 - acc: 0.4900 - val_loss: 0.9686 - val_acc: 0.5000
Epoch 8/50
1000

In [1]:
print("Hello")

Hello
